In [ ]:
!kaggle competitions download -c digit-recognizer

digit-recognizer.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip digit-recognizer.zip

Archive:  digit-recognizer.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train.dtypes

,0
label,int64
pixel0,int64
pixel1,int64
pixel2,int64
pixel3,int64
...,...
pixel779,int64
pixel780,int64
pixel781,int64
pixel782,int64


In [ ]:
test.dtypes

,0
pixel0,int64
pixel1,int64
pixel2,int64
pixel3,int64
pixel4,int64
...,...
pixel779,int64
pixel780,int64
pixel781,int64
pixel782,int64


In [ ]:
train.shape

(42000, 785)

In [ ]:
test.shape

(28000, 784)

In [ ]:
print('Missing values in training data : ' + str(train.isnull().values.sum()))
print('Missing values in testing data : ' + str(test.isnull().values.sum()))

Missing values in training data : 0
Missing values in testing data : 0


In [ ]:
X = train.drop('label', axis=1)
y = train['label']

In [ ]:
X = X / 255.0
test = test / 255.0

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# ***Logistic Regression***

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# Initialize and train model
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

In [ ]:
# Predict and evaluate
y_pred_lr = lr_model.predict(X_val)
print("Logistic Regression Accuracy:", accuracy_score(y_val, y_pred_lr))

Logistic Regression Accuracy: 0.9142857142857143


In [ ]:
y_pred_test = lr_model.predict(test)

In [ ]:
submission = pd.DataFrame({
    "ImageId": list(range(1, len(y_pred_test)+1)),
    "Label": y_pred_test
})
submission.to_csv("submission.csv", index=False)

# ***Convolution Neural Networks (CNN)***

In [25]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical

# Reshape data for CNN: (samples, height, width, channels)
X_train_cnn = X_train.values.reshape(-1, 28, 28, 1)
X_val_cnn = X_val.values.reshape(-1, 28, 28, 1)
X_test_cnn = test.values.reshape(-1, 28, 28, 1)


# One-hot encode labels
y_train_cat = to_categorical(y_train, num_classes=10)
y_val_cat = to_categorical(y_val, num_classes=10)

# Build CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train CNN
cnn_model.fit(X_train_cnn, y_train_cat, epochs=10, validation_data=(X_val_cnn, y_val_cat))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 42s 33ms/step - accuracy: 0.8314 - loss: 0.5234 - val_accuracy: 0.9752 - val_loss: 0.0714
Epoch 2/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 48s 41ms/step - accuracy: 0.9693 - loss: 0.0986 - val_accuracy: 0.9826 - val_loss: 0.0576
Epoch 3/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 73s 33ms/step - accuracy: 0.9787 - loss: 0.0710 - val_accuracy: 0.9879 - val_loss: 0.0404
Epoch 4/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 41s 33ms/step - accuracy: 0.9835 - loss: 0.0541 - val_accuracy: 0.9886 - val_loss: 0.0409
Epoch 5/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 41s 33ms/step - accuracy: 0.9858 - loss: 0.0444 - val_accuracy: 0.9895 - val_loss: 0.0316
Epoch 6/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 40s 32ms/step - accuracy: 0.9885 - loss: 0.0370 - val_accuracy: 0.9898 - val_loss: 0.0363
Epoch 7/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 40s 32ms/step - accuracy: 0.9900 - loss: 0.0332 - val_accuracy: 0.9900 - val_loss: 0.0355
Epoch 8/10
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 40s 31ms/step - accuracy: 0.9911 -

In [28]:
# Evaluate model on training data
train_loss, train_accuracy = cnn_model.evaluate(X_train_cnn, y_train_cat, verbose=0)

# Print training accuracy
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")


Training Accuracy: 99.79%
